1. Write a approx 750-800 (3PAGE) word description of: 
    - Chatbot application (e.g., use-case, domain of operation).
    - Atleast 3 different test cases that clearly illustrate chatbot behavior.
    - How you connected your work to lecture material 
    - Advanced techniques that you used

If you are submitting a PDF, please mention that here. 

-----------------------------------------------------
##### PASTE YOUR TEXT HERE
-----------------------------------------------------

------------
2. CODE
-----------

In [22]:
# PASTE THE CODE 
# Make this easy for user testing - Running this cell should load all code for Chatbot.




In [1]:
# ATLEAST 3 INTERESTING RESULTS - AND FUNCTION CALLS TO GENERATE THEM


3. Reflect on project management (approx 250 words) 
    - Timeliness: Reflect on how consistently you made an effort meet deadlines
    - Organization: Reflect on how managed the code-base as the project size increased

-----------------------------------------------------

Reflection goes here

-----------------------------------------------------


4. Process reflection (approx 200 words) - Discuss the week by week iterative development of your chatbot.
    - Describe each week of chatbot devleopment.
    - What was the feedback you received? How did you work on the feedback to improve chatbot? What new features did you add? 

-----------------------------------------------------

Reflection goes here

-----------------------------------------------------
